In [23]:
import pyomo.environ as pe  # Pyomo environment
from idaes.core import FlowsheetBlock, StateBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95

# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.properties = iapws95.Iapws95ParameterBlock()

# Provided data in Celsius, convert to Kelvin
T_hot_inlet_C = 662.30
T_hot_outlet_C = 192.60
T_cold_inlet_C = 20.00
T_cold_outlet_C = 496.94

T_hot_inlet = T_hot_inlet_C + 273.15  # Convert to Kelvin
T_hot_outlet = T_hot_outlet_C + 273.15  # Convert to Kelvin
T_cold_inlet = T_cold_inlet_C + 273.15  # Convert to Kelvin
T_cold_outlet = T_cold_outlet_C + 273.15  # Convert to Kelvin

# Assume constant Cp for gases (ideal gas assumption)
Cp_hot = 1.04  # Specific heat of nitrogen (N2) in kJ/kg·K
Cp_cold = 1.005  # Specific heat of air in kJ/kg·K
Cp_steam = 4.18  # Specific heat of steam in kJ/kg·K (assumed for the example)

# Reference temperature (for simplicity, assume 0°C = 273.15 K)
T_ref = 273.15

# Calculate enthalpy using Cp (simplified)
def calc_enthalpy(Cp, T):
    return Cp * (T - T_ref)

# Calculate enthalpy for the hot and cold streams
h_hot_inlet = calc_enthalpy(Cp_hot, T_hot_inlet)  # Inlet enthalpy for the hot stream (N2/steam)
h_cold_inlet = calc_enthalpy(Cp_cold, T_cold_inlet)  # Inlet enthalpy for the cold stream (air)

# Create the Heat Exchanger model
model.fs.heat_exchanger = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

# Fixing inlet flow and pressure values (assuming standard conditions)
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(100)  # Hot stream flow rate (mol/s)
model.fs.heat_exchanger.shell_inlet.pressure.fix(101325)  # Standard atmospheric pressure
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(100)  # Cold stream flow rate (mol/s)
model.fs.heat_exchanger.tube_inlet.pressure.fix(101325)  # Standard atmospheric pressure

# Fixing enthalpies for the heat exchanger (we are using simplified calculation)
model.fs.heat_exchanger.shell_inlet.enth_mol[0].set_value(h_hot_inlet)
model.fs.heat_exchanger.tube_inlet.enth_mol[0].set_value(h_cold_inlet)

# Fixing heat exchanger parameters
model.fs.heat_exchanger.area.fix(1000)  # Heat exchanger area (m², fixed for now)
model.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(100)  # Overall heat transfer coefficient (W/m²·K)

# Initialize the model
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

# Now we can calculate the outlet temperatures and enthalpies after solving the model.
# Calculate the outlet enthalpy for hot and cold streams
T_hot_outlet_optimized = pe.value(model.fs.heat_exchanger.shell.properties_out[0].temperature)
h_hot_outlet = calc_enthalpy(Cp_hot, T_hot_outlet_optimized)

T_cold_outlet_optimized = pe.value(model.fs.heat_exchanger.tube.properties_out[0].temperature)
h_cold_outlet = calc_enthalpy(Cp_cold, T_cold_outlet_optimized)

# Output the enthalpies and temperatures
print(f"Hot Stream Inlet Enthalpy: {h_hot_inlet} kJ/mol")
print(f"Cold Stream Inlet Enthalpy: {h_cold_inlet} kJ/mol")
print(f"Hot Stream Outlet Enthalpy: {h_hot_outlet} kJ/mol")
print(f"Cold Stream Outlet Enthalpy: {h_cold_outlet} kJ/mol")
print(f"Hot Stream Inlet Temperature: {T_hot_inlet_C} °C")
print(f"Hot Stream Outlet Temperature: {T_hot_outlet_C} °C")
print(f"Cold Stream Inlet Temperature: {T_cold_inlet_C} °C")
print(f"Cold Stream Outlet Temperature: {T_cold_outlet_C} °C")


2025-03-28 12:06:14 [INFO] idaes.init.fs.heat_exchanger: Initialization Completed, optimal - <undefined>
Hot Stream Inlet Enthalpy: 688.792 kJ/mol
Cold Stream Inlet Enthalpy: 20.099999999999998 kJ/mol
Hot Stream Outlet Enthalpy: 0.9048712431338026 kJ/mol
Cold Stream Outlet Enthalpy: 8.49325608546755 kJ/mol
Hot Stream Inlet Temperature: 662.3 °C
Hot Stream Outlet Temperature: 192.6 °C
Cold Stream Inlet Temperature: 20.0 °C
Cold Stream Outlet Temperature: 496.94 °C


In [25]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 

# Assign a value to the heat exchanger area before visualization
model.fs.heat_exchanger.area.set_value(pe.value(model.fs.heat_exchanger.area))

# Visualize the model
visualize(model.fs, "HeatExchangerAreaMinimization")

2025-03-28 12:07:29 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-03-28 12:07:29 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchangerAreaMinimization.json'
2025-03-28 12:07:29 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerAreaMinimization.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-03-28 12:07:30 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:57304/app?id=HeatExchangerAreaMinimization


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x000002217E6B7070>, port=57304, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x000002217E6B6350>)